In [15]:
import pickle
import torch
import dgl
import matplotlib.pyplot as plt
import networkx as nx
from classes import *
from torch.utils.data import DataLoader
import pickle
from torch.utils.tensorboard import SummaryWriter
import torch.nn.functional as F
import numpy as np
from dgl import DGLGraph
from mpl_toolkits.mplot3d import Axes3D
from knn import *
from matplotlib import animation
import math



In [16]:

data_path = '../data/small_move_cube_data.pickle'
use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')
with open(data_path,"rb") as f:
  dataa = pickle.load(f)


model_path = 'models/st8.pth'
net = stNet()
net.load_state_dict(torch.load(model_path))
net.eval()

TypeError: ord() expected a character, but string of length 0 found

In [ ]:
i=20
features = dataa[i]["coor"] ##lsit of 20
seen = dataa[i]["visible"] ##lsit of 20
newfeatures = []
for framecoor in features:
    newfeatures.append(np.hstack((framecoor,np.zeros((588,1)))))
for framenum in range(len(newfeatures)):                        
    for idx in range(newfeatures[framenum].shape[0]):
       if idx not in seen[framenum]:
           newfeatures[framenum][idx] = np.array([0,0,0,1])

temporal_g= temporal_edges(20)
spatial_g= knn_graph_edges(8,20)
tg = DGLGraph(temporal_g)
sg = DGLGraph(spatial_g)
features1 = torch.Tensor(np.vstack(features))
newfeatures = torch.Tensor(np.vstack(newfeatures))



In [ ]:

# fig = plt.figure(figsize=(10,10))
# ax = fig.add_subplot(111, projection='3d')
# xs = features[:,0]
# ys = features[:,1]
# zs = features[:,2]
# colors = (features-features.min(axis=0))/(features.max(axis=0)-features.min(axis=0))
# ax.set_title('Truth')

# for j in range(list(g.edges()[0].size())[0]):
#     node1 = g.edges()[0][j]
#     node2 = g.edges()[1][j]
#     x = np.array((features[node1][0], features[node2][0]))
#     y = np.array((features[node1][1], features[node2][1]))
#     z = np.array((features[node1][2], features[node2][2]))

#     # Plot the connecting lines
#     ax.plot(x, y, z, c='black', alpha=0.5)
# elev = 20
# azim = 125
# # elev = 0
# # azim = 0
# ax.view_init(elev=elev, azim=azim)
# ax.scatter3D(xs,ys,zs, c=colors,edgecolors='k', alpha=0.7,s=70 )
# plt.show()


In [ ]:
# fig = plt.figure(figsize=(8,8))
# ax = fig.add_subplot(111, projection='3d')
# k=1
# ax.set_title('{} nearest'.format(k))
# def init():
#     ax.scatter3D(xs,ys,zs, c=colors,edgecolors='k', alpha=0.7,s=70 )

#     for j in range(list(g.edges()[0].size())[0]):
#         node1 = g.edges()[0][j]
#         node2 = g.edges()[1][j]
#         x = np.array((features[node1][0], features[node2][0]))
#         y = np.array((features[node1][1], features[node2][1]))
#         z = np.array((features[node1][2], features[node2][2]))

#         # Plot the connecting lines
#         ax.plot(x, y, z, c='black', alpha=0.5)
#     return fig

# def animate(i):
#     ax.view_init(elev=10., azim=i*4)
#     return fig

# # Animate
# anim = animation.FuncAnimation(fig, animate, init_func=init,
#                                frames=90, interval=20)
# print("lol")
# # Save
# # anim.save('basic_animation.mp4', writer = "ffmpeg",fps=30, extra_args=['-vcodec', 'libx264'])

# print("lol2")
# # In[46]:


# plt.rcParams["animation.html"] = "jshtml"
# anim

In [ ]:
featurest = torch.Tensor(newfeatures)   #.to(device=device, non_blocking=True)
outputt = net(sg,tg,featurest)
output = outputt.cpu().detach().numpy()

In [ ]:
framennum = 0
fig = plt.figure(figsize=(18,7))
ax = fig.add_subplot(121, projection='3d')
output = np.reshape(output,(588,3,-1))
frame = output[:,:,framenum]
X = frame[:,0]
Y = frame[:,1]
Z = frame[:,2]
temp = np.array([X+Y+Z])
temp = (temp-temp.min())/(temp.max()-temp.min())
colors = np.hstack((temp.T,np.zeros((588,2))+0.5))

for idx,col in enumerate(output):
    if idx not in dataa[i]["visible"][framenum]:
        colors[idx] = np.array([0,1,0])


ax.set_title('Output')
ax2 = fig.add_subplot(122, projection='3d')
frame = features[framenum]
xs = frame[:,0]
ys = frame[:,1]
zs = frame[:,2]


temp = (temp-temp.min())/(temp.max()-temp.min())
colors2 = np.hstack((temp.T,np.zeros((588,2))+0.5))
for idx,col in enumerate(output):
    if idx not in dataa[i]["visible"][framenum]:
        colors2[idx] = np.array([0,1,0])
ax2.set_title('Truth')
# ax2.set_xlim(3.4,5)

campos = dataa[i]['cam']
print(campos)
r= (campos[0]**2+ campos[1]**2+ campos[2]**2)**0.5
rou = math.acos(campos[2]/r)
theta = math.asin(campos[1]/(r*math.sin(rou)))
elev = rou
azim = theta

# elev = -30
# azim = 125

ax.view_init(elev=elev, azim=azim)
ax2.view_init(elev=elev, azim=azim)
ax.scatter3D(X, Y, Z, c=colors )
ax2.scatter3D(xs, ys, zs,c=colors2)
loss = F.l1_loss(outputt,features1).item()
plt.show()
print("L1Loss:{}".format(loss))



In [ ]:
fig = plt.figure(figsize=(18,7))
ax = fig.add_subplot(121, projection='3d')
ax.set_title('Output')
ax2 = fig.add_subplot(122, projection='3d')
ax2.set_title('Truth')
# ax.set_ylim(-0.8,0.8)

def init():
    ax.scatter3D(X, Y, Z, c=colors )
    ax2.scatter3D(xs, ys, zs,c=colors2)
    return fig

def animate(i):
    ax.view_init(elev=rou, azim=i*4)
    ax2.view_init(elev=rou, azim=i*4)
    return fig

# Animate
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=90, interval=20)
print("lol")
# Save
anim.save('basic_animation.mp4', writer = "ffmpeg",fps=30, extra_args=['-vcodec', 'libx264'])

print("lol2")
# In[46]:


plt.rcParams["animation.html"] = "jshtml"
anim

In [ ]:
fig = plt.figure(figsize=(18,7))
ax = fig.add_subplot(121, projection='3d')
ax.set_title('Output')
ax2 = fig.add_subplot(122, projection='3d')
ax2.set_title('Truth')
# ax.set_ylim(-0.8,0.8)
graph=ax.scatter3D(X, Y, Z, c=colors )
graph2=ax2.scatter3D(xs, ys, zs,c=colors2)
print(type(graph))

def init():
    graph=ax.scatter3D(X, Y, Z, c=colors )
    graph2=ax2.scatter3D(xs, ys, zs,c=colors2)
    ax.view_init(elev=rou, azim=i*4)
    ax2.view_init(elev=rou, azim=i*4)
    return fig

def animate(thi):
    plt.cla()
    framenum=thi%20
    frame = output[:,:,framenum]
    X = frame[:,0]
    Y = frame[:,1]
    Z = frame[:,2]
    temp = np.array([X+Y+Z])
    temp = (temp-temp.min())/(temp.max()-temp.min())
    colors = np.hstack((temp.T,np.zeros((588,2))+0.5))

    for idx,col in enumerate(output):
        if idx not in dataa[i]["visible"][framenum]:
            colors[idx] = np.array([0,1,0])
    frame = features[framenum]
    xs = frame[:,0]
    ys = frame[:,1]
    zs = frame[:,2]


    temp = (temp-temp.min())/(temp.max()-temp.min())
    colors2 = np.hstack((temp.T,np.zeros((588,2))+0.5))
    for idx,col in enumerate(output):
        if idx not in dataa[i]["visible"][framenum]:
            colors2[idx] = np.array([0,1,0])
    ax = fig.add_subplot(121, projection='3d')
    ax.set_title('Output')
    ax2 = fig.add_subplot(122, projection='3d')
    ax2.set_title('Truth')
    # ax.set_ylim(-0.8,0.8)
    graph=ax.scatter3D(X, Y, Z, c=colors )
    graph2=ax2.scatter3D(xs, ys, zs,c=colors2)
    return fig

# Animate
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=20, interval=20)
print("lol")
# Save
anim.save('basic_animation.mp4', writer = "ffmpeg",fps=10, extra_args=['-vcodec', 'libx264'])

print("lol2")
# In[46]:


plt.rcParams["animation.html"] = "jshtml"
anim